<a href="https://colab.research.google.com/github/ARAVIND-00/GenAI_practise/blob/main/HYbrid_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_core langchain_community rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.9 [186 kB]
Fetched 186 kB in 2s (120 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.9_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.9) ...
Setting up poppler-utils (22.02.0-2ubuntu0.9) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import huggingface_hub
from langchain.retrievers import BM25Retriever,EnsembleRetriever

In [10]:
filepath='/content/orca.pdf'

document=PyPDFLoader(filepath).load()
document

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-06T00:53:20+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-06T00:53:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/orca.pdf', 'total_pages': 51, 'page': 0, 'page_label': '1'}, page_content='Orca: Progressive Learning from Complex\nExplanation Traces of GPT-4\nSubhabrata Mukherjee∗†, Arindam Mitra∗\nGanesh Jawahar, Sahaj Agarwal, Hamid Palangi, Ahmed Awadallah\nMicrosoft Research\nAbstract\nRecent research has focused on enhancing the capability of smaller models\nthrough imitation learning, drawing on the outputs generated by large\nfoundation models (LFMs). A number of issues impact the quality of these\nmodels, ranging from limited imitation signals from shallow LFM outputs;\nsmall scale homogeneous training data; and most not

In [12]:
document[0].page_content

'Orca: Progressive Learning from Complex\nExplanation Traces of GPT-4\nSubhabrata Mukherjee∗†, Arindam Mitra∗\nGanesh Jawahar, Sahaj Agarwal, Hamid Palangi, Ahmed Awadallah\nMicrosoft Research\nAbstract\nRecent research has focused on enhancing the capability of smaller models\nthrough imitation learning, drawing on the outputs generated by large\nfoundation models (LFMs). A number of issues impact the quality of these\nmodels, ranging from limited imitation signals from shallow LFM outputs;\nsmall scale homogeneous training data; and most notably alack of rigorous\nevaluation resulting in overestimating the small model’s capability as they\ntend to learn to imitate the style, but not the reasoning process of LFMs. To\naddress these challenges, we develop Orca, a 13-billion parameter model\nthat learns to imitate the reasoning process of LFMs. Orca learns from\nrich signals from GPT-4 including explanation traces; step-by-step thought\nprocesses; and other complex instructions, guided 

In [14]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

chunks=splitter.split_documents(document)
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-06T00:53:20+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-06T00:53:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/orca.pdf', 'total_pages': 51, 'page': 0, 'page_label': '1'}, page_content='Orca: Progressive Learning from Complex\nExplanation Traces of GPT-4\nSubhabrata Mukherjee∗†, Arindam Mitra∗\nGanesh Jawahar, Sahaj Agarwal, Hamid Palangi, Ahmed Awadallah\nMicrosoft Research\nAbstract\nRecent research has focused on enhancing the capability of smaller models\nthrough imitation learning, drawing on the outputs generated by large\nfoundation models (LFMs). A number of issues impact the quality of these\nmodels, ranging from limited imitation signals from shallow LFM outputs;\nsmall scale homogeneous training data; and most not